In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pathlib
import itertools
import pandas.util.testing as tm

%matplotlib inline

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [7]:
cwd =os.getcwd()
cwd

'/content'

In [5]:
path = pathlib.Path('C:\\GP_2019\\Mestrado\\Saneamento\\Dados\\Bases\\Desagregado_geral')
os.chdir(path)

FileNotFoundError: ignored

In [4]:
df = pd.read_csv('Modelo0909_custo_No_NA.csv', sep = ';', encoding = 'utf-8',engine='c')

FileNotFoundError: ignored